En esta entrega deberia poder evaluar el score que me dio el algoritmo entrenado, pero al meter tan pocas variables y con ellas querer predecir algo tan dificil como la EDAD del participante involucrado, no llego a tener una precision que se pueda exponer. Con lo cual estoy haciendo un grave UNDERFITTING, para solucionar esto, en la reentrega de este punto, voy a categorizar las demas variables (que considero importantes) que no estoy usando, de manera inteligente para que el algoritmo las pueda utilizar efectivamente.

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

In [2]:
siniestros = 'victimas-siniestros.csv'
sin_df = pd.read_csv(siniestros)

C:\Users\Sistemas\AppData\Local\Temp\ipykernel_2744\1239150759.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  sin_df = pd.read_csv(siniestros)


VEO CANTIDAD DE NULOS POR COLUMNA

In [3]:
sin_df.isnull().sum()

causa                               0
rol                               335
tipo                             1100
sexo                              197
edad                              559
mes                                37
periodo                             0
fecha                              61
hora                               17
lugar_hecho                      3716
direccion_normalizada             765
tipo_calle                          0
direccion_normalizada_arcgis      772
calle1                            765
altura                          26123
calle2                           8247
codigo_calle                     9290
codigo_cruce                    12525
geocodificacion                  1095
semestre                            0
x                                1095
y                                1095
geom                             1095
cantidad_victimas                   0
comuna                           1098
geom_3857                        1095
tipo_colisio

PRIMERO PARA REALIZAR EL FEATURE SELECTION VOY A LIMPIAR EL DATASET

In [4]:
sin_df.value_counts('causa')

causa
lesiones     32795
homicidio      439
dtype: int64

In [5]:
sin_df['comuna'].fillna(0,inplace=True)
sin_df['comuna'] = sin_df['comuna'].astype(int)

SACO LAS 37 FILAS CON LA COLUMNA 'mes' CON UN VALOR NULO 

In [6]:
sin_df.dropna(subset=['mes','hora'],inplace=True)

RELLENO LOS VALORES NULOS DE OTRAS COLUMNAS

In [7]:
sin_df['tipo_colision1'].fillna('sin_data',inplace=True)
sin_df['tipo'].fillna('sin_data',inplace=True)
sin_df['rol'].fillna('sin_data',inplace=True)
sin_df['sexo'].fillna('sin_data',inplace=True)
sin_df['calle1'].fillna('sin_data',inplace=True)
sin_df['participantes_acusados'].fillna('sin_data',inplace=True)
sin_df['participantes_victimas'].fillna('sin_data',inplace=True)
sin_df['codigo_calle'].fillna(-1,inplace=True)
sin_df['x'].fillna(-1,inplace=True)
sin_df['y'].fillna(-1,inplace=True)
sin_df['edad'].fillna(-1,inplace=True)
sin_df['geom_3857'].fillna(-1,inplace=True)
#sin_df['hora'] = sin_df['hora'].to_datetime(inplace=True)


In [8]:
#def simplificar_tipo(ser):
#    return ser

#def simplificar_tipocolision(ser):
#    return ser

In [9]:
#sin_df['tipo_simpl'] = sin_df['tipo'].apply(simplificar_tipo)
#sin_df['tipo_col_simpl'] = sin_df['tipo_colision1'].apply(simplificar_tipo)

Todavia no lo hice, pero arriba tengo que simplificar estas columnas en 4 o 5 categorias

In [156]:
sin_df = sin_df[['edad','tipo_colision1','tipo','causa','rol','sexo','codigo_calle','calle1','x','y','geom_3857','mes','hora','comuna','participantes_acusados','participantes_victimas']]
sin_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33181 entries, 0 to 33233
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   edad                    33181 non-null  float64
 1   tipo_colision1          33181 non-null  object 
 2   tipo                    33181 non-null  object 
 3   causa                   33181 non-null  object 
 4   rol                     33181 non-null  object 
 5   sexo                    33181 non-null  object 
 6   codigo_calle            33181 non-null  float64
 7   calle1                  33181 non-null  object 
 8   x                       33181 non-null  float64
 9   y                       33181 non-null  float64
 10  geom_3857               33181 non-null  object 
 11  mes                     33181 non-null  float64
 12  hora                    33181 non-null  object 
 13  comuna                  33181 non-null  int32  
 14  participantes_acusados  33181 non-null

CONSEGUI UN DATASET CON LAS COLUMNAS QUE CONSIDERO MAS IMPORTANTES, SIN VALORES NULOS

Ahora voy a entrenar mi algoritmo para predecir la EDAD de la persona involucrada, basandose en las otras 13 columnas elegidas.

In [10]:
sin_df = sin_df[['edad','codigo_calle','x','y','mes','comuna']]

In [12]:
X = sin_df.drop("edad", axis=1)
y = sin_df["edad"]
#X = pd.get_dummies(X, columns=['tipo_colision1','tipo','rol','sexo','codigo_calle','calle1','x','y','geom_3857','mes','hora','comuna','participantes_acusados','participantes_victimas'])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
reg = LinearRegression().fit(X_train, y_train)

In [15]:
y_pred = reg.predict(X_test)

In [16]:
score = reg.score(X_test, y_test)
print("Precisión del modelo: {:.2f}%".format(score * 100))

Precisión del modelo: 0.23%


HICE LA PRUEBA DE ENTRENAR UN MODELO SOLO CON 3 COLUMNAS NUMERICAS PARA COMPROBAR QUE EL ALGORITMO FUNCIONA; AHORA RESTARIA LIMPIAR Y CATEGORIZAR LAS VARIABLES PARA MEJORAR LA PRECISION

In [165]:
#X = sin_df.drop("causa", axis=1)
#y = sin_df["causa"]

In [166]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#TENGO QUE HACER QUE LAS COLUMNAS SEAN CATEGORICAS O NUMERICAS

In [ ]:
#clf = DecisionTreeClassifier().fit(X_train, y_train)

In [ ]:
#y_pred = clf.predict(X_test)

In [ ]:
#score = clf.score(X_test, y_test)
#print("Precisión del modelo: {:.2f}%".format(score * 100))

NO LLEGUE A HACER LAS CATEGORIAS PARA ESTA ENTREGA POR LO CUAL NO TENGO UN ALGORITMO ENTRENADO QUE ME DE UN VALOR APRECIABLE DE SCORE 